### this notebook is experiment to dive into the news & contents, the relationship between stock performance. Values could be how long the influence and what's the impact or some predictive values.

* Clean data set
* Design regression
* Output summarize

In [5]:
import pandas as pd
import numpy as np
import datetime

base_dir = "../"
wsjR = pd.read_excel(f"{base_dir}WSJ_Articles_AA_Filtered.xlsx")
wsj = wsjR
wsj.head(3)

,news_time,news_title,content,url,keywords
0,2013-01-04 08:52:00,Decision on AMR-US Airways Merger Is 'Weeks' Away,The chief executive of American Airlines' pare...,http://www.wsj.com/articles/DJFDBR0020130104e9...,[]
1,2013-01-07 11:17:00,US Airways Pilots Union Board Approves AMR Mer...,As AMR Corp.'s American Airlines and its credi...,http://www.wsj.com/articles/DJFHYW0020130107e9...,[]
2,2013-01-09 08:47:00,AMR's Shares May Hold Some Value After Bankruptcy,"American Airlines parent AMR Corp., which has ...",http://www.wsj.com/articles/DJFDBR0020130109e9...,[]


In [ ]:
# clean data set, leave only 
print(wsj.info())
wsj = wsjR.dropna(subset=["news_time", "news_title"], how='all')
del(wsjR)
wsj = wsj[~wsj.news_time.isnull()]
wsj

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5662 entries, 0 to 5661
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   news_time   5662 non-null   datetime64[ns]
 1   news_title  5662 non-null   object        
 2   content     2521 non-null   object        
 3   new_time    5662 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 350.2+ KB
None


,news_time,news_title,content,new_time
0,2021-07-22 12:44:00.000000,\n Airline Stocks Shouldn’t All Look the Sa...,No two airlines are the same: Each operates in...,"July 22, 2021 12:44 pm ET"
1,2021-07-21 09:00:00.000000,Summer Travel Seasons Don’t Get Weirder Than T...,Some 2.2 million travelers waded through U.S. ...,"July 21, 2021 9:00 am ET"
2,2021-07-20 17:04:59.999999,\n Stocks Rebound After Dramatic Selloff\n,"U.S. stock prices rose sharply, with the S&P 5...","July 20, 2021 5:05 pm ET"
3,2021-07-20 05:29:59.999999,\n Supply-Chain Executives See Long Road Ah...,Logistics companies and other supply-chain ope...,"July 20, 2021 5:30 am ET"
4,2021-07-19 18:43:00.000000,\n We’ll Always Have Covid\n,Will the pandemic ever end? That’s what invest...,"July 19, 2021 6:43 pm ET"
...,...,...,...,...
5657,2013-01-07 11:17:00.000000,US Airways Pilots Union Board Approves AMR Mer...,NaN,"Jan. 7, 2013 11:17 am ET"
5658,2013-01-07 09:11:00.000000,Elpida Creditors Navigate New Distressed Inves...,NaN,"Jan. 7, 2013 9:11 am ET"
5659,2013-01-07 09:11:00.000000,Distressed Investors Find Value in Post-Bankru...,NaN,"Jan. 7, 2013 9:11 am ET"
5660,2013-01-07 09:10:00.000001,Disgruntled Creditors Seek Examiners to Gain L...,NaN,"Jan. 7, 2013 9:10 am ET"


In [7]:
# use the "nltk.sent_tokenize" method to extract sentences from the text and "nltk.word_tokenize" to tokenize words in a sentence.
# clean headline and content
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def base_clean(parags):
    if (isinstance(parags, float)):
        return parags
    if len(parags) > 1:
        parags = re.sub(r'•', '', parags)
        parags = re.sub(r'|', '', parags)
    return parags

def paras_raw_text(parags_list):
    # tokenize, lemma, stop words
    
    all_words = list()
    lists = []
    r_count=0
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for i in range(len(parags_list)):
        paragR = parags_list[i]
        if (str(paragR) == 'nan') | (isinstance(paragR,float)):
            # jump solely float
            lists.append(np.nan)
            continue
        r_count+=1
        if (r_count%1000==0):
            print (r_count)
        parag = base_clean(paragR)

        sentences = nltk.sent_tokenize(parag) # works for multiple sentences
        parag_nonstop = ""
        for sentence in sentences:
            sent_tok = nltk.word_tokenize(sentence)
            #remove stopwords
            #float doesn't have sentiment
            sent_tok = [lemmatizer.lemmatize(word) for word in sent_tok 
                        if (word not in stopwords_list and len(word) > 1 and type(word) == str)]
#             print ("Original=["+sentence+"]")
            #create a tokenized version of the sentence:
            sent_tok_str = " ".join(word for word in sent_tok)
#             print ("Tokenized=["+sent_tok_str+"]")

            parag_nonstop += sent_tok_str
            parag_nonstop += " "

            #create a list of all tokenized words across all sentences:
            for word in sent_tok:
                all_words.append(word.lower())

        lists.append(parag_nonstop)
    return all_words, lists

data = wsj.copy()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#load the nltk default English stopwords list:
stopwords_list = nltk.corpus.stopwords.words('english')


all_words, headline_lists = paras_raw_text(data["news_title"].values)
all_words2, content_lists = paras_raw_text(data["content"].values)
all_words.append(all_words2)
data["clean_headline"] = headline_lists
data["clean_content"] = content_lists
data


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Haotian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haotian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Haotian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Haotian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haotian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1000
2000
3000
1000
2000
3000


,news_time,news_title,content,url,keywords,clean_headline,clean_content
0,2013-01-04 08:52:00,Decision on AMR-US Airways Merger Is 'Weeks' Away,The chief executive of American Airlines' pare...,http://www.wsj.com/articles/DJFDBR0020130104e9...,[],Decision AMR-US Airways Merger Is 'Weeks Away,The chief executive American Airlines parent s...
1,2013-01-07 11:17:00,US Airways Pilots Union Board Approves AMR Mer...,As AMR Corp.'s American Airlines and its credi...,http://www.wsj.com/articles/DJFHYW0020130107e9...,[],US Airways Pilots Union Board Approves AMR Mer...,As AMR Corp. 's American Airlines creditor con...
2,2013-01-09 08:47:00,AMR's Shares May Hold Some Value After Bankruptcy,"American Airlines parent AMR Corp., which has ...",http://www.wsj.com/articles/DJFDBR0020130109e9...,[],AMR 's Shares May Hold Some Value After Bankru...,American Airlines parent AMR Corp. reorganizin...
3,2013-01-09 15:39:00,AMR Bondholders to Weigh US Airways Merger,American Airlines bondholders this week signed...,http://www.wsj.com/articles/DJFDBR0020130109e9...,[],AMR Bondholders Weigh US Airways Merger,American Airlines bondholder week signed confi...
4,2013-01-09 17:18:00,AMR Bondholders Sign Confidentiality Agreement...,American Airlines bondholders this week signed...,http://www.wsj.com/articles/DJFHYW0020130109e9...,[],AMR Bondholders Sign Confidentiality Agreement...,American Airlines bondholder week signed confi...
...,...,...,...,...,...,...,...
3497,2021-07-20 05:30:00,Supply-Chain Executives See Long Road Ahead fo...,Logistics companies and other supply-chain ope...,https://www.wsj.com/articles/supply-chain-exec...,[],Supply-Chain Executives See Long Road Ahead Di...,Logistics company supply-chain operator steppi...
3498,2021-07-20 17:05:00,Stocks Rebound After Dramatic Selloff,"U.S. stock prices rose sharply, with the S&P 5...",https://www.wsj.com/articles/global-stock-mark...,[],Stocks Rebound After Dramatic Selloff,U.S. stock price rose sharply 500 recording be...
3499,2021-07-21 09:00:00,Summer Travel Seasons Don’t Get Weirder Than T...,Some 2.2 million travelers waded through U.S. ...,https://www.wsj.com/articles/summer-travel-sea...,[],Summer Travel Seasons Don Get Weirder Than Thi...,Some 2.2 million traveler waded U.S. airport s...
3500,2021-07-22 12:44:00,Airline Stocks Shouldn’t All Look the Same,No two airlines are the same: Each operates in...,https://www.wsj.com/articles/airline-stocks-sh...,[],Airline Stocks Shouldn All Look Same,No two airline Each operates different market ...


In [15]:
# import tqdm
data["clean_text"] = data["clean_headline"].astype(str) + " " + data["clean_content"].astype(str)
file2 = open(r"../AAL_text.txt","w+")
for content in data["clean_text"]:
    # print(content)
    file2.write(str(content.encode('utf-8')))
    file2.write("\n")
    file2.write("\n")
file2.close()

In [ ]:
# sentiment text
from nltk.sentiment import SentimentIntensityAnalyzer
def process_sentiment(df, list_columns):
    # input: series, return pos, neg series of sentiment scores
    sia = SentimentIntensityAnalyzer()
    for column in list_columns:
        print(column)
        df[column+"_pos"]=[sia.polarity_scores(str(x))['pos'] for x in df[column]]
        df[column+"_neg"]=[sia.polarity_scores(str(x))['neg'] for x in df[column]]
        df[column+"_compound"]=[sia.polarity_scores(str(x))['compound'] for x in df[column]]
    return df

data = process_sentiment(data, ["clean_headline", "clean_content"])
data.head(3)

clean_headline
clean_content


,news_time,news_title,content,new_time,clean_headline,clean_content,clean_headline_pos,clean_headline_neg,clean_headline_compound,clean_content_pos,clean_content_neg,clean_content_compound
0,2021-07-22 12:44:00.000000,\n Airline Stocks Shouldn’t All Look the Sa...,No two airlines are the same: Each operates in...,"July 22, 2021 12:44 pm ET",Airline Stocks Shouldn All Look Same,No two airline Each operates different market ...,0.0,0.000,0.000,0.131,0.106,0.7645
1,2021-07-21 09:00:00.000000,Summer Travel Seasons Don’t Get Weirder Than T...,Some 2.2 million travelers waded through U.S. ...,"July 21, 2021 9:00 am ET",Summer Travel Seasons Don Get Weirder Than Thi...,Some 2.2 million traveler waded U.S. airport s...,0.0,0.158,-0.128,0.065,0.129,-0.9905
2,2021-07-20 17:04:59.999999,\n Stocks Rebound After Dramatic Selloff\n,"U.S. stock prices rose sharply, with the S&P 5...","July 20, 2021 5:05 pm ET",Stocks Rebound After Dramatic Selloff,U.S. stock price rose sharply 500 recording be...,0.0,0.000,0.000,0.144,0.048,0.9959


In [ ]:
#         if not vs['neg'] > 0.1:
#             if vs['pos']-vs['neg'] > 0:
#                 pos_correct += 1
#             pos_count +=1

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
% matplotlib inline
# lower max_font_size, change the maximum number of word and lighten the background:
text = " ".join(data["clean_headline"]+data["clean_content"])
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
# calculate daily news return
# each day 30 minutes and after day return
# judge whether there is news in the time slot and it's sentiment

sp500_itd = intraday.iloc[:,:4]
aal_itd = intraday.iloc[:,4:8]
sp500_itd.dropna(how="all"), aal_itd.dropna(how="all")

(          Date      Time  SP 500 Price  Volume
 0     20210723  19:30:00         21.07   17641
 1     20210723  19:00:00         21.11    9968
 2     20210723  18:30:00         21.12    4402
 3     20210723  18:00:00         21.15    9274
 4     20210723  17:30:00         21.16   22782
 ...        ...       ...           ...     ...
 1851  20210503  06:00:00         21.77     667
 1852  20210503  05:30:00         21.77    2262
 1853  20210503  05:00:00         21.77    3470
 1854  20210503  04:30:00         21.85    1963
 1855  20210503  04:00:00         21.84    3279
 
 [1856 rows x 4 columns],         Date.1    Time.1  AAL Price  Volume.1
 0     20210723  19:30:00      21.07     17641
 1     20210723  19:00:00      21.11      9968
 2     20210723  18:30:00      21.12      4402
 3     20210723  18:00:00      21.15      9274
 4     20210723  17:30:00      21.16     22782
 ...        ...       ...        ...       ...
 1851  20210503  06:00:00      21.77       667
 1852  20210503  05:3

In [ ]:
# iterate each date, calculate two returns
# three returns (positive return, neg return, middle return)


In [ ]:
# rank by compound score
data.loc[data.news_time > pd.to_datetime('2021-05-05')].sort_values(by=['clean_content_compound'], ascending=False)

,news_time,news_title,content,new_time,clean_headline,clean_content,clean_headline_pos,clean_headline_neg,clean_headline_compound,clean_content_pos,clean_content_neg,clean_content_compound
93,2021-05-28 08:02:00.000000,I Thought I Was Done With Iraq. Then a Fellow ...,I remember the day Lance Corporal Walter O’Hai...,"May 28, 2021 8:02 am ET",Thought Was Done With Iraq Then Fellow Marine ...,remember day Lance Corporal Walter Haire kille...,0.000,0.000,0.0000,0.179,0.086,0.9997
117,2021-05-05 08:59:00.000000,Why Airline Rewards Programs Are Trying Harder...,"Status matters: social, work, financial, healt...","May 5, 2021 8:59 am ET",Why Airline Rewards Programs Are Trying Harder...,Status matter social work financial health leg...,0.437,0.000,0.7351,0.203,0.037,0.9989
33,2021-07-09 12:34:00.000000,\n Americans Can Travel to Greece Now. Here...,"MY FIRST NIGHT in Mykonos, a few weeks ago, th...","July 9, 2021 12:34 pm ET",Americans Can Travel Greece Now Here How Smoot...,MY FIRST NIGHT Mykonos week ago whitewashed hi...,0.000,0.000,0.0000,0.140,0.039,0.9989
48,2021-06-29 13:22:00.000000,\n United Airlines Bets on Post-Pandemic Gr...,United Airlines Holdings Inc. UAL -0.88...,"June 29, 2021 1:22 pm ET",United Airlines Bets Post-Pandemic Growth With...,United Airlines Holdings Inc. UAL -0.88 making...,0.403,0.000,0.6597,0.152,0.025,0.9982
29,2021-07-09 17:15:00.000000,\n What’s in Biden’s Big Business Executive...,President Biden’s executive order lays out a p...,"July 9, 2021 5:15 pm ET",What Biden Big Business Executive Order,President Biden executive order lay plan addre...,0.000,0.000,0.0000,0.156,0.068,0.9981
...,...,...,...,...,...,...,...,...,...,...,...,...
1,2021-07-21 09:00:00.000000,Summer Travel Seasons Don’t Get Weirder Than T...,Some 2.2 million travelers waded through U.S. ...,"July 21, 2021 9:00 am ET",Summer Travel Seasons Don Get Weirder Than Thi...,Some 2.2 million traveler waded U.S. airport s...,0.000,0.158,-0.1280,0.065,0.129,-0.9905
19,2021-07-14 09:00:00.000000,\n How CEOs Think the Covid Crisis Will Sha...,To run an airline is to manage crisis. And for...,"July 14, 2021 9:00 am ET",How CEOs Think Covid Crisis Will Shape Flying,To run airline manage crisis And two longest-s...,0.000,0.369,-0.6249,0.119,0.163,-0.9909
99,2021-05-25 05:26:00.000000,\n Belarus Flight Diversion Worries Airline...,The forced diversion of a Ryanair H...,"May 25, 2021 5:26 am ET",Belarus Flight Diversion Worries Airline Indus...,The forced diversion Ryanair Holdings RYAAY 0....,0.000,0.259,-0.4215,0.085,0.152,-0.9933
85,2021-06-02 09:00:00.000000,"\n In Venezuela, Covid-19 Data Is a State S...","On most days, Venezuela’s authoritarian govern...","June 2, 2021 9:00 am ET",In Venezuela Covid-19 Data Is State Secret Cit...,On day Venezuela authoritarian government repo...,0.000,0.000,0.0000,0.062,0.173,-0.9983
